In [35]:
alphabet = bytes('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/', 'ascii')
alphabet

b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'

In [ ]:
bytes()

In [119]:
# base64 字节编码（课堂方法）
def base64encode(src):
    ret = bytearray()
    if isinstance(src, str):
        _src = src.encode() #  默认utf-8
    elif isinstance(src, bytes):
        _src = src
    else:
        raise TypeError
        
    lenght = len(_src)
    for i in range(0, lenght, 3):
        #print(_src)
        triple = _src[i: i+3]
        r = 3 - len(triple)
        if r:
            triple += b'\x00' * r
        b = int.from_bytes(triple, 'big')
        
        for j in range(18, -1 ,-6):
            index = b >> j & 0x3f if j != 18 else b >> j
            #print(index)
            ret.append(alphabet[index])
    if r:       
        ret[-r:] = b'=' * r
    return ret.decode()

In [124]:
import base64
base64.b64encode(b'~').decode()

'fg=='

In [103]:
base64encode('abc')

b'abc'


'YWJj'

In [104]:
base64encode('~')

b'~'


'fg=='

In [116]:
base64encode('a')

b'a'


'YQ=='

In [120]:
str1 = 'Man is distinguished, not only by his reason, but by this singular passion from other animals, which is a lust of the mind, that by a perseverance of delight in the continued and indefatigable generation of knowledge, exceeds the short vehemence of any carnal pleasure.'
str2 = 'Man is distinguished'
base64encode(str1)


'TWFuIGlzIGRpc3Rpbmd1aXNoZWQsIG5vdCBvbmx5IGJ5IGhpcyByZWFzb24sIGJ1dCBieSB0aGlzIHNpbmd1bGFyIHBhc3Npb24gZnJvbSBvdGhlciBhbmltYWxzLCB3aGljaCBpcyBhIGx1c3Qgb2YgdGhlIG1pbmQsIHRoYXQgYnkgYSBwZXJzZXZlcmFuY2Ugb2YgZGVsaWdodCBpbiB0aGUgY29udGludWVkIGFuZCBpbmRlZmF0aWdhYmxlIGdlbmVyYXRpb24gb2Yga25vd2xlZGdlLCBleGNlZWRzIHRoZSBzaG9ydCB2ZWhlbWVuY2Ugb2YgYW55IGNhcm5hbCBwbGVhc3VyZS4='

In [157]:
b'\x60'.decode()

'`'

In [ ]:
# 实现缓存和过期清楚的功能

In [114]:
from functools import wraps
import inspect
import time
import datetime

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()        
        ret = fn(*args, **kwargs)
        delta = (datetime.datetime.now() - start).total_seconds()
        print(fn.__name__, delta)
        return ret
    return wrapper
    
def mag_cache(duration=2):
    def cache(fn):
        cache_dic = {}
        @wraps(fn)
        def wrapper(*args, **kwargs):
            
            def _clear_expire(cache):
                expire_keys = []
                for k, (_, v) in cache.items():
                    now = datetime.datetime.now().timestamp()
                    if now - v > duration:
                        expire_keys.append(k)
                for i in expire_keys:
                    cache.pop(i) 
            _clear_expire(cache_dic)

            
            def _make_key():
                sig = inspect.signature(fn)
                params = sig.parameters

                target = dict(zip(list(params.keys()), args), **kwargs)
                print(target)
                for k, v in params.items():
                    if k not in target:
                        target[k] = v.default
                print(target)
                return tuple(sorted(target.items())), target
            key, target = _make_key()
            
            
            if key not in cache_dic:
                cache_dic[key] = fn(**target), datetime.datetime.now().timestamp()

            return cache_dic[key]

        return wrapper
    return cache
    # add = logger(mag_cache(wrapper)) logger里面的fn是mag_cache(wrapper)
           # add里面的fn 指logger里面的wrapper
@logger             # add = mag_cache(2)(add)->mag_cache(cache)->mag_cache(wrapper)
@mag_cache(20)              # add里面的fn 是add函数
def add(a=4, b=5):
    time.sleep(2)
    return a+b

In [122]:
add(6)

{'a': 6}
{'a': 6, 'b': 5}
add 0.000368


(11, 1566969101.009297)

In [123]:
from functools import wraps
import inspect
import time
import datetime

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()        
        ret = fn(*args, **kwargs)
        delta = (datetime.datetime.now() - start).total_seconds()
        print(fn.__name__, delta)
        return ret
    return wrapper
    
def mag_cache(duration=2):
    def cache(fn):
        cache_dic = {}
        
        def _clear_expire(cache):
            expire_keys = []
            for k, (_, v) in cache.items():
                now = datetime.datetime.now().timestamp()
                if now - v > duration:
                    expire_keys.append(k)
            for i in expire_keys:
                cache.pop(i)
        
        def _make_key(fn, args, kwargs):
            sig = inspect.signature(fn)
            params = sig.parameters

            target = dict(zip(list(params.keys()), args), **kwargs)
            # print(params)
            for k, v in params.items():
                if k not in target:
                    target[k] = v.default                
            return tuple(sorted(target.items())), target
                   
        @wraps(fn)
        def wrapper(*args, **kwargs):
            _clear_expire(cache_dic)
            key, target = _make_key(fn, args, kwargs)
                        
            if key not in cache_dic:
                cache_dic[key] = fn(**target), datetime.datetime.now().timestamp()

            return cache_dic[key]

        return wrapper
    return cache
    # add = logger(mag_cache(wrapper)) logger里面的fn是mag_cache(wrapper)
           # add里面的fn 指logger里面的wrapper
@logger             # add = mag_cache(2)(add)->mag_cache(cache)->mag_cache(wrapper)
@mag_cache(20)              # add里面的fn 是add函数
def add(a=4, b=5):
    time.sleep(2)
    return a+b

In [125]:
print(add(4))
print('********')
print(add(4))
print(add(4, 5))
print(add(a=4, b=5))
print(add(a=4))
print(add(b=5))
print(add(4, b=5))

add 2.005203
(9, 1566969229.202248)
********
add 0.000141
(9, 1566969229.202248)
add 0.000117
(9, 1566969229.202248)
add 9.5e-05
(9, 1566969229.202248)
add 9.1e-05
(9, 1566969229.202248)
add 5.8e-05
(9, 1566969229.202248)
add 5.9e-05
(9, 1566969229.202248)


In [129]:
from functools import wraps
import inspect
import time
import datetime

def _clear_expire(cache, duration):
    expire_keys = []
    for k, (_, v) in cache.items():
        now = datetime.datetime.now().timestamp()
        if now - v > duration:
            expire_keys.append(k)
    for i in expire_keys:
        cache.pop(i)
        
def _make_key(fn, args, kwargs):
    sig = inspect.signature(fn)
    params = sig.parameters

    target = dict(zip(list(params.keys()), args), **kwargs)
    # print(params)
    for k, v in params.items():
        if k not in target:
            target[k] = v.default                
    return tuple(sorted(target.items())), target
                           

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()        
        ret = fn(*args, **kwargs)
        delta = (datetime.datetime.now() - start).total_seconds()
        print(fn.__name__, delta)
        return ret
    return wrapper
    
def mag_cache(duration=2):
    def cache(fn):
        cache_dic = {}
               
        @wraps(fn)
        def wrapper(*args, **kwargs):
            _clear_expire(cache_dic, duration)
            key, target = _make_key(fn, args, kwargs)
                        
            if key not in cache_dic:
                cache_dic[key] = fn(**target), datetime.datetime.now().timestamp()

            return cache_dic[key]

        return wrapper
    return cache
    # add = logger(mag_cache(wrapper)) logger里面的fn是mag_cache(wrapper)
           # add里面的fn 指logger里面的wrapper
@logger             # add = mag_cache(2)(add)->mag_cache(cache)->mag_cache(wrapper)
@mag_cache(20)              # add里面的fn 是add函数
def add(a=4, b=5):
    time.sleep(2)
    return a+b

In [130]:
print(add(4))
print('********')
print(add(4))
print(add(4, 5))
print(add(a=4, b=5))
print(add(a=4))
print(add(b=5))
print(add(4, b=5))

add 2.003159
(9, 1566969654.319878)
********
add 7.4e-05
(9, 1566969654.319878)
add 4.3e-05
(9, 1566969654.319878)
add 3.9e-05
(9, 1566969654.319878)
add 5.6e-05
(9, 1566969654.319878)
add 4.4e-05
(9, 1566969654.319878)
add 5.1e-05
(9, 1566969654.319878)


In [131]:
from functools import wraps
import inspect
import time
import datetime

def _clear_expire(cache, duration):
    expire_keys = []
    for k, (_, v) in cache.items():
        now = datetime.datetime.now().timestamp()
        if now - v > duration:
            expire_keys.append(k)
    for i in expire_keys:
        cache.pop(i)
        
def _make_key(fn, args, kwargs):
    sig = inspect.signature(fn)
    params = sig.parameters

    target = dict(zip(list(params.keys()), args), **kwargs)

    target.update(((k, params[k].default) for k in (params.keys() - target.keys())))

        
    return tuple(sorted(target.items())), target
                           

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()        
        ret = fn(*args, **kwargs)
        delta = (datetime.datetime.now() - start).total_seconds()
        print(fn.__name__, delta)
        return ret
    return wrapper
    
def mag_cache(duration=2):
    def cache(fn):
        cache_dic = {}
               
        @wraps(fn)
        def wrapper(*args, **kwargs):
            _clear_expire(cache_dic, duration)
            key, target = _make_key(fn, args, kwargs)
                        
            if key not in cache_dic:
                cache_dic[key] = fn(**target), datetime.datetime.now().timestamp()

            return cache_dic[key]

        return wrapper
    return cache
    # add = logger(mag_cache(wrapper)) logger里面的fn是mag_cache(wrapper)
           # add里面的fn 指logger里面的wrapper
@logger             # add = mag_cache(2)(add)->mag_cache(cache)->mag_cache(wrapper)
@mag_cache(20)              # add里面的fn 是add函数
def add(a=4, b=5):
    time.sleep(2)
    return a+b

In [132]:
print(add(4))
print('********')
print(add(4))
print(add(4, 5))
print(add(a=4, b=5))
print(add(a=4))
print(add(b=5))
print(add(4, b=5))

add 2.000258
(9, 1566969897.635582)
********
add 0.000148
(9, 1566969897.635582)
add 0.000101
(9, 1566969897.635582)
add 0.000119
(9, 1566969897.635582)
add 0.000104
(9, 1566969897.635582)
add 9.4e-05
(9, 1566969897.635582)
add 6e-05
(9, 1566969897.635582)


In [189]:
from functools import wraps
import inspect
import time
import datetime
from collections import OrderedDict

def _clear_expire(cache, duration):
    expire_keys = []
    for k, (_, v) in cache.items():
        now = datetime.datetime.now().timestamp()
        if now - v > duration:
            expire_keys.append(k)
    for i in expire_keys:
        cache.pop(i)
        
def _make_key(fn, *args, **kwargs):
    sig = inspect.signature(fn)
    params = sig.parameters
#     print('params.keys():', params.keys())
#     print('args:', args)
    print(params)
    print(params.keys(), type(params.keys()))
    target = OrderedDict(zip(params.keys(), args))
#     print('***:', target)
    for k, v in params.items():
        if k not in target:
            if k in kwargs:
                target[k] = kwargs[k]
            else:
                target[k] = v.default
#     print(target)
    return tuple(target.items()), target
                           

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()        
        ret = fn(*args, **kwargs)
        delta = (datetime.datetime.now() - start).total_seconds()
        print(fn.__name__, delta)
        return ret
    return wrapper
    
def mag_cache(duration=2):
    def cache(fn):
        cache_dic = {}
               
        @wraps(fn)
        def wrapper(*args, **kwargs):
            _clear_expire(cache_dic, duration)
            key, target = _make_key(fn, *args, **kwargs)
            print(key)          
            if key not in cache_dic:
                cache_dic[key] = fn(**target), datetime.datetime.now().timestamp()

            return cache_dic[key]

        return wrapper
    return cache
    # add = logger(mag_cache(wrapper)) logger里面的fn是mag_cache(wrapper)
           # add里面的fn 指logger里面的wrapper
@logger             # add = mag_cache(2)(add)->mag_cache(cache)->mag_cache(wrapper)
@mag_cache(20)              # add里面的fn 是add函数
def add(a=4, b=5):
    time.sleep(2)
    return a+b

In [190]:
print(add(4))
print('********')
print(add(4))
print(add(4, 5))
print(add(a=4, b=5))
print(add(a=4))
print(add(b=5))
print(add(4, b=5))

OrderedDict([('a', <Parameter "a=4">), ('b', <Parameter "b=5">)])
odict_keys(['a', 'b']) <class 'odict_keys'>
(('a', 4), ('b', 5))
add 2.003614
(9, 1566977973.586199)
********
OrderedDict([('a', <Parameter "a=4">), ('b', <Parameter "b=5">)])
odict_keys(['a', 'b']) <class 'odict_keys'>
(('a', 4), ('b', 5))
add 0.000417
(9, 1566977973.586199)
OrderedDict([('a', <Parameter "a=4">), ('b', <Parameter "b=5">)])
odict_keys(['a', 'b']) <class 'odict_keys'>
(('a', 4), ('b', 5))
add 0.000343
(9, 1566977973.586199)
OrderedDict([('a', <Parameter "a=4">), ('b', <Parameter "b=5">)])
odict_keys(['a', 'b']) <class 'odict_keys'>
(('a', 4), ('b', 5))
add 0.00029
(9, 1566977973.586199)
OrderedDict([('a', <Parameter "a=4">), ('b', <Parameter "b=5">)])
odict_keys(['a', 'b']) <class 'odict_keys'>
(('a', 4), ('b', 5))
add 0.000257
(9, 1566977973.586199)
OrderedDict([('a', <Parameter "a=4">), ('b', <Parameter "b=5">)])
odict_keys(['a', 'b']) <class 'odict_keys'>
(('a', 4), ('b', 5))
add 0.000219
(9, 156697797

In [186]:
a = OrderedDict()
a.update([('b', 3), ('a', 2)])
a

OrderedDict([('b', 3), ('a', 2)])

In [188]:
a = OrderedDict()
a.update({'b':1, 'a':2})
a

OrderedDict([('b', 1), ('a', 2)])

In [175]:
params = {'a':1, 'b':2}
args = (3,)
target = OrderedDict(zip(params.keys(), args))
target

OrderedDict([('a', 3)])

In [147]:
a = {}
a.update(a=1, b=2)
print(a)

{'a': 1, 'b': 2}


In [151]:
# , {'6':'w', '7':'q'}
target = dict(zip(list(range(3)), ['a', 'b']), c=9, d=8, **{'6':'w', '7':'q'})
target

{0: 'a', 1: 'b', 'c': 9, 'd': 8, '6': 'w', '7': 'q'}

In [138]:
for i in zip(list(range(3)), ['a', 'b']):
    print(i)

(0, 'a')
(1, 'b')


In [64]:
x = {'a': 1}
def f1():
    def fn(y):
        return y
    x = fn(x)
    return x

In [54]:
x = {'a': 1}
def fn(y):
    return y
x = {'b': 1}
x = fn(x)

In [14]:
a = 1
def fn():
    a = 2
    return a
a = fn()
a

2

In [39]:
# l = []
def f1():
#     print(1)
    def f2():
#         print(l)
        l.append(1)
        return l
    l = f2()
    
    return l


In [40]:
f1()

[1]

In [28]:
a = []

def f():
    a.append(1)
    a = list()
    
f()

UnboundLocalError: local variable 'a' referenced before assignment

In [ ]:
from functools import wraps
import inspect
import time
import datetime

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()        
        ret = fn(*args, **kwargs)
        delta = (datetime.datetime.now() - start).total_seconds()
        print(fn.__name__, delta)
        return ret
    return wrapper
    
def mag_cache(duration=2):
    def cache(fn):
        cache_dic = {}
        @wraps(fn)
        def wrapper(*args, **kwargs):
            
            def _clear_expire():
                expire_keys = []
                for k, (_, v) in cache_dic.items():
                    now = datetime.datetime.now().timestamp()
                    if now - v > duration:
                        expire_keys.append(k)
                for i in expire_keys:
                    cache_dic.pop(i)
                return cache_dic   
            cache_dic = _clear_expire()
            
            def _make_key():
                sig = inspect.signature(fn)
                params = sig.parameters

                target = dict(zip(list(params.keys()), args), **kwargs)
                # print(params)
                for k, v in params.items():
                    if k not in target:
                        target[k] = v.default
                key = tuple(sorted(target.items()))
                return key
            key = _make_key()
            
            
            if key not in cache_dic:
                cache_dic[key] = fn(**kwargs), datetime.datetime.now().timestamp()

            return cache_dic[key]

        return wrapper
    return cache
    # add = logger(mag_cache(wrapper)) logger里面的fn是mag_cache(wrapper)
           # add里面的fn 指logger里面的wrapper
@logger             # add = mag_cache(2)(add)->mag_cache(cache)->mag_cache(wrapper)
@mag_cache(2)              # add里面的fn 是add函数
def add(a=4, b=5):
    time.sleep(2)
    return a+b

In [181]:
c = OrderedDict([('a', <Parameter "a=4">), ('b', <Parameter "b=5">)])
for i, j in c:
    pass

SyntaxError: invalid syntax (<ipython-input-181-55a12776396a>, line 1)

In [171]:
a = {'b':1, 'a':2}
b = sorted(a.items()) # sorted可以对<class 'dict_items'>排序
# 并且返回一个list [('a', 2), ('b', 1)]
print(a.items(), type(a.items()))
print(b)
print(tuple(b))



dict_items([('b', 1), ('a', 2)]) <class 'dict_items'>
[('a', 2), ('b', 1)]
(('a', 2), ('b', 1))


In [168]:
sorted()

SyntaxError: invalid syntax (<ipython-input-168-363e88e975b4>, line 1)

In [154]:
b = {'b':2}
b.update(a=1,c=1)
print(b)

{'b': 2, 'a': 1, 'c': 1}


In [155]:
dict(a=1, b=2, c=3)

{'a': 1, 'b': 2, 'c': 3}

In [150]:
print({'a':1})

{'a': 1}


In [71]:
hex(int.from_bytes('~'.encode(), 'big'))

'0x7e'

In [59]:
hex(int.from_bytes(b'abc', 'big'))

'0x616263'

In [ ]:
# 0110 0001

In [28]:
97 >> 4

6

In [100]:
a = [1,2]
a[-0:] = [0] * 2
print(a)


[0, 0]


In [31]:
# 右移动几位代表2的几次幂
97 >> 0b0100, 97 >> 0x4, 97 >> 4

(6, 6, 6)

In [83]:
hex(int.from_bytes(b'a' + b'\x00', 'big'))

'0x6100'

In [88]:
b'\x00'.decode(), b'\x00', 0x00

('\x00', b'\x00', 0)

0

In [90]:
int.from_bytes(b'\x00', 'big')

0

In [91]:
bytes('\x00', 'ascii')

b'\x00'

In [92]:
bytes('\x00', 'gbk')

b'\x00'

In [93]:
bytes('\x00', 'utf-8')

b'\x00'

In [96]:
bytes(range(97, 100))

b'abc'

In [2]:
import time

In [3]:

cachedic = {}
def makekey(*args, **kwargs):
    key = args
    if kwargs:
        key += (object(), )
        for item in kwargs.items():
            key += item
    return key

def cache(fn):
    def wrapper(*args, **kwargs):
        key = makekey(*args, **kwargs)
        if key in cachedic:
            return cachedic[key]
        ret = fn(*args, **kwargs)
        cachedic[key] = ret
        print('cachedic: ',cachedic)
        return ret
    return wrapper

@cache
def add(a, b):
    time.sleep(2)
    return a + b

In [4]:
add(1, 2)

cachedic:  {(1, 2): 3}


3

In [5]:
add(1, 2)

3

In [6]:
add(a=1, b=2)

cachedic:  {(1, 2): 3, (<object object at 0x110046c00>, 'a', 1, 'b', 2): 3}


3

In [225]:
import inspect, time
from collections import OrderedDict

cachedic = {}
# def makekey(*args, **kwargs):
#     keyset = {}
#     keyset.update(args)
#     for item in kwargs.items():
#         keyset.update(item)
#     return keyset
# def makedefaultkey():
#     pass
def updatedic(fn, args, kwargs):
    tempdic = OrderedDict()
    for k, v in inspect.signature(fn).parameters.items():
        tempdic[k] = v.default
    if args:
        lenght = len(args)
        if lenght == 1:
            tempdic.update(dict(a=args[0]))
        else:
            tempdic.update(dict(a=args[0]))
            tempdic.update(dict(b=args[1]))
    if kwargs:
        tempdic.update(kwargs)
    return tempdic
    

def cache(fn):
    def wrapper(*args, **kwargs):
        #setkey = makekey(*args, **kwargs)
        
#         if key in cachedic:
#             return cachedic[key]
        tempdic = updatedic(fn, args, kwargs)
    
        print('cachedic:', cachedic)
        
        if tuple(tempdic.items()) in cachedic:
            return cachedic[tuple(tempdic.items())]
        
        
        
        ret = fn(*args, **kwargs)
        cachedic[tuple(tempdic.items())] = ret
#         print(inspect.signature(fn))
#         print(type(inspect.signature(fn)))
        
        #cachedic[key] = ret
        #print('cachedic: ',cachedic)
        return ret
    return wrapper

@cache # add = cache(add)
def add(a=4, b=5):
    time.sleep(2)
    return a + b

In [226]:
add()


cachedic: {}


9

In [227]:
add(a=4)

cachedic: {(('a', 4), ('b', 5)): 9}


9

In [228]:
add(4)

cachedic: {(('a', 4), ('b', 5)): 9}


9

In [229]:
add(4, b=5)

cachedic: {(('a', 4), ('b', 5)): 9}


9

In [230]:
add(4, 5)

cachedic: {(('a', 4), ('b', 5)): 9}


9

In [231]:
add(a=4, b=5)

cachedic: {(('a', 4), ('b', 5)): 9}


9

In [187]:
tempdic = {}
tempdic.update(dict(a=1))
print(tempdic)

{'a': 1}


In [193]:
tuple({'a':1, 'b':2}.items())

(('a', 1), ('b', 2))

In [186]:
tuple([['a', 1], ['b', 2]])

(['a', 1], ['b', 2])

In [14]:
str1 = 'abc'
str1.encode('utf8')

b'abc'

In [18]:
for i in range(97, 100):
    print(bin(i))

0b1100001
0b1100010
0b1100011


In [113]:
import math
mapping = {}
for i in range(62):
    if i < 26:
        mapping[i] = chr(i+65)
    elif i < 52:
        mapping[i] = chr(i+71)
    else:
        mapping[i] = chr(i-4)
mapping[62] = '+'
mapping[63] = '/'
print(mapping)

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'a', 27: 'b', 28: 'c', 29: 'd', 30: 'e', 31: 'f', 32: 'g', 33: 'h', 34: 'i', 35: 'j', 36: 'k', 37: 'l', 38: 'm', 39: 'n', 40: 'o', 41: 'p', 42: 'q', 43: 'r', 44: 's', 45: 't', 46: 'u', 47: 'v', 48: 'w', 49: 'x', 50: 'y', 51: 'z', 52: '0', 53: '1', 54: '2', 55: '3', 56: '4', 57: '5', 58: '6', 59: '7', 60: '8', 61: '9', 62: '+', 63: '/'}


In [85]:
def convert_str_to_base64(str1):
    #chrlst = ['A', 'b', '+']
    binlst = ['{}{}'.format('0', bin(ord(i))[2:]) for i in str1]
    #print(binlst)
    sumstr = ''.join(binlst)
    # print(sumstr)
    lenght = math.ceil(len(sumstr)/6)
    # print(lenght)
    rightlst = ['{:0<8}'.format('00'+sumstr[6*i: 6*(i+1)]) for i in range(lenght)]
    #print(rightlst)
    newstr = ''.join([mapping.get(int('0b'+i[2:], 2), None) for i in rightlst])
    return newstr

In [78]:
convert_str_to_base64('ab+')

['01100001', '01100010', '0101011']
['00011000', '00010110', '00001001', '00010110']


'YWJW'

In [80]:
convert_str_to_base64('A')

['01000001']
['00010000', '00010000']


'QQ'

In [86]:
str1 = 'Man is distinguished, not only by his reason, but by this singular passion from other animals, which is a lust of the mind, that by a perseverance of delight in the continued and indefatigable generation of knowledge, exceeds the short vehemence of any carnal pleasure.'

convert_str_to_base64(str1)

'TWFuQNLmgZGlzdGluZ3Vpc2hlZFiBub3RA3tzY8oGJ5QNDS5oHJlYXNvbliBidXRAxPKB0aGlzQObS3M7q2MLkgcGFzc2lvbkDM5N7agb3RoZXJAwtzS2sLY5rEDu0NLG0IGlzQMKBsdXN0QN7MgdGhlQNrS3MixA6NDC6IGJ5QMKBwZXJzZXZlcmFuY2VA3syBkZWxpZ2h0QNLcgdGhlQMbe3OjS3OrKyIGFuZEDS3MjKzMLo0s7CxNjKgZ2VuZXJhdGlvbkDezIGtub3dsZWRnZViBleGNlZWRzQOjQyoHNob3J0QOzK0MraytzGyoG9mQMLc8oGNhcm5hbEDg2MrC5urkyrg'

In [108]:
def patch_bin(str2):   
    new_bin_str = ''.join(['{:0>8}'.format(bin(ord(i))[2:]) for i in str2])
    length = len(new_bin_str)
    if length == 24:
        return new_bin_str
    return '{:0<24}'.format(new_bin_str)

In [109]:
def split_bin(str3):   
    base64 = ''.join([mapping.get(int(str3[6*i:6*(i+1)], 2), None) for i in range(4)])
    return base64

In [110]:
def convert_last_str(str4, index):
    num = 0
    last_str = str4[index+3:]
    print('last_str:', last_str)
    last_bin = ''.join(['{:0>8}'.format(bin(ord(i))[2:]) for i in last_str])
    print('last_bin:', last_bin)
    lenght = len(last_bin)
    if 0 < lenght <=8:
        num = 2
        last_bin = '{:0<12}'.format(last_bin)
    elif 8 < lenght <= 16:
        num =1
        last_bin = '{:0<18}'.format(last_bin)
    base64 = ''.join([mapping.get(int(last_bin[i:i+6], 2), None) for i in range(0, lenght, 6)])
    print('**:', base64)
    if num == 1:
        base64 += '='
    if num == 2:
        base64 += 2 * '='
    return base64
          

In [111]:
def convert_str_to_base64(str1):
    lenght = len(str1)
    beforebase = ''
    for i in range(0, lenght-3, 3):
        #print(str1[i:i+3])
        base64 = split_bin(patch_bin(str1[i:i+3])) 
        beforebase += base64
    #print(beforebase)
    base64 = beforebase + convert_last_str(str1, i)
    print(convert_last_str(str1, i))
    return base64
    

In [114]:
str1 = 'Man is distinguished, not only by his reason, but by this singular passion from other animals, which is a lust of the mind, that by a perseverance of delight in the continued and indefatigable generation of knowledge, exceeds the short vehemence of any carnal pleasure.'

convert_str_to_base64(str1)

last_str: e.
last_bin: 0110010100101110
**: ZS4
last_str: e.
last_bin: 0110010100101110
**: ZS4
ZS4=


'TWFuIGlzIGRpc3Rpbmd1aXNoZWQsIG5vdCBvbmx5IGJ5IGhpcyByZWFzb24sIGJ1dCBieSB0aGlzIHNpbmd1bGFyIHBhc3Npb24gZnJvbSBvdGhlciBhbmltYWxzLCB3aGljaCBpcyBhIGx1c3Qgb2YgdGhlIG1pbmQsIHRoYXQgYnkgYSBwZXJzZXZlcmFuY2Ugb2YgZGVsaWdodCBpbiB0aGUgY29udGludWVkIGFuZCBpbmRlZmF0aWdhYmxlIGdlbmVyYXRpb24gb2Yga25vd2xlZGdlLCBleGNlZWRzIHRoZSBzaG9ydCB2ZWhlbWVuY2Ugb2YgYW55IGNhcm5hbCBwbGVhc3VyZS4='

In [ ]:
TWFuIGlzIGRpc3Rpbmd1aXNoZWQsIG5vdCBvbmx5IGJ5IGhpcyByZWFzb24sIGJ1dCBieSB0aGlz
IHNpbmd1bGFyIHBhc3Npb24gZnJvbSBvdGhlciBhbmltYWxzLCB3aGljaCBpcyBhIGx1c3Qgb2Yg
dGhlIG1pbmQsIHRoYXQgYnkgYSBwZXJzZXZlcmFuY2Ugb2YgZGVsaWdodCBpbiB0aGUgY29udGlu
dWVkIGFuZCBpbmRlZmF0aWdhYmxlIGdlbmVyYXRpb24gb2Yga25vd2xlZGdlLCBleGNlZWRzIHRo
ZSBzaG9ydCB2ZWhlbWVuY2Ugb2YgYW55IGNhcm5hbCBwbGVhc3VyZS4=

In [57]:
a = [1,2,3,4,5,6,7,8,9,10]
a[8:15]

[9, 10]

In [59]:
import math
math.ceil(13/6)

3

In [61]:
int('0b01000', 2)

1

In [93]:
for i in range(2):
    print(i)
print(i)

0
1
1


In [124]:
int('111000', 2)

56

In [15]:
int('0b011000', 2)

24

In [91]:
bin(ord('n'))

'0b1101110'

In [8]:
int.from_bytes('abc'.encode(), 'big')

6382179

In [ ]:
int.from_bytes()

In [81]:
type(b'a')

bytes

In [82]:
b'a'[1:]

b''

In [1]:
'abc'.encode()

b'abc'

In [3]:
bytes('abc', 'ascii')

b'abc'

In [4]:
'小猫'.encode()

b'\xe5\xb0\x8f\xe7\x8c\xab'

In [ ]:
str.encode()

In [9]:
bin(65)

'0b1000001'

In [10]:
a = bytearray()

In [11]:
a

bytearray(b'')

In [17]:
a.append(97)
a

bytearray(b'\x02\x10a')

In [13]:
a

bytearray(b'\x02')